# Attribute-based Text Generation from Amazon Product Reviews

## Library Imports

In [ ]:
from numpy import zeros, concatenate, asarray
from IPython.display import display, HTML

## Notification JavaScript snippets

In [ ]:
def browser_alert(message):
    display(HTML('<script type="text/javascript">alert("' + message + '");</script>'))
    
def browser_notify(message):
    display(HTML('<script type="text/javascript">var notification=new Notification("' + \
                 'Jupyter Notification",{icon:"http://blog.jupyter.org/content/' + \
                 'images/2015/02/jupyter-sq-text.png",body:"' + message + \
                 '"});</script>'))

In [ ]:
browser_notify("test")

## Reading Data

In [ ]:
dataset_path = "/home/v2john/attr-reviews-dataset/"

In [ ]:
dev_set_path = dataset_path + "dev.txt"
train_set_path = dataset_path + "train.txt"
test_set_path = dataset_path + "test.txt"
vocab_path = dataset_path + "vocab.txt"

In [ ]:
def build_vocab_index(vocab_file_path):
    
    vocab_index = dict()
    current_index = 0
    
    with open(vocab_file_path) as vocab_file:
        for line in vocab_file:
            word = line.split()[0]
            vocab_index[word] = current_index
            current_index += 1
            
    return vocab_index


def build_user_and_product_indices(review_file_path):
    
    user_index = dict()
    product_index = dict()
    
    current_user_index = 0
    current_product_index = 0
    
    with open(review_file_path) as review_file:
        for line in review_file:
            split_line = line.split()
            user_id = split_line[0]
            product_id = split_line[1]
            
            if user_id not in user_index:
                user_index[user_id] = current_user_index
                current_user_index += 1
                
            if product_id not in product_index:
                product_index[product_id] = current_product_index
                current_product_index += 1
                
    return user_index, product_index


def build_rating_index():
    
    rating_index = dict()
    
    for i in range(0, 5):
        rating_index[(i + 1) * 1.0] = i
        
    return rating_index

In [ ]:
vocab_index = build_vocab_index(vocab_path)
num_words = len(vocab_index)
print(str(num_words) + " terms in the vocabulary")

In [ ]:
user_index, product_index = build_user_and_product_indices(dev_set_path)
num_users = len(user_index)
num_products = len(product_index)
print(str(num_users) + " distinct users")
print(str(num_products) + " distinct products")

In [ ]:
rating_index = build_rating_index()
num_ratings = len(rating_index)
print(str(num_ratings) + " distinct ratings")

In [ ]:
browser_notify("Indices built")

## Convert build attribute and text-embedding versions of the training data

In [ ]:
from nltk import word_tokenize

In [ ]:
def update_term_matrix(term_index, term, term_rows, term_cols, term_data, index):
        
        
    term_data.append(1)
    term_rows.append(index)
    term_cols.append(term_index[term])

In [ ]:
def get_review_sequence(review, vocab_index):
    
    review_sequence = list()
    tokens = word_tokenize(review)
    
    for token in tokens:
        try:
            word_index = update_term_matrix(vocab_index, num_words, str(token))
            review_sequence.append(word_index)
        except KeyError:
            pass  # skip missing keys
    
    return asarray(review_sequence)

def get_attribute_vectors_and_text_embedding_sequences(review_file_path):

    x_train_att = list()
    x_train_rev = list()
    i = 0
    user_rows = list()
    user_cols = list()
    user_data = list()
    product_rows = list()
    product_cols = list()
    product_data = list()
    rating_rows = list()
    rating_cols = list()
    rating_data = list()
    
    with open(review_file_path) as review_file:
        for line in review_file:
            split_line = line.split("\t")
            user_id = split_line[0]
            product_id = split_line[1]
            rating = float(split_line[2])
            review_text = split_line[3]
            
            update_term_matrix(user_index, user_id, user_rows, user_cols, user_data, i),
            update_term_matrix(product_index, product_id, product_rows, product_cols, product_data, i),
            update_term_matrix(rating_index, rating, rating_rows, rating_cols, rating_data, i)
            
            x_train_rev.append(get_review_sequence(review_text, vocab_index))
            
    return asarray(x_train_att), asarray(x_train_rev)

In [ ]:
x_train_att, x_train_rev = get_attribute_vectors_and_text_embedding_sequences(dev_set_path)

In [ ]:
x_train_att.shape, x_train_att[0].shape

In [ ]:
x_train_rev.shape, x_train_rev[0].shape

### Save training attributes

In [ ]:
import pickle

In [ ]:
flatfiles_path = "/home/v2john/text-gen-flatfiles/"

In [ ]:
# # Save vectors

# with open(flatfiles_path + 'x_train_att.pkl', 'wb') as x_train_att_file:
#     pickle.dump(x_train_att, x_train_att_file)

# with open(flatfiles_path + 'x_train_rev.pkl', 'wb') as x_train_rev_file:
#     pickle.dump(x_train_rev, x_train_rev_file)

In [ ]:
# Restore vectors

with open(flatfiles_path + 'x_train_att.pkl', 'rb') as x_train_att_file:
    x_train_att = pickle.load(x_train_att_file)
    
with open(flatfiles_path + 'x_train_rev.pkl', 'rb') as x_train_rev_file:
    x_train_rev = pickle.load(x_train_rev_file)

In [ ]:
x_train_att.shape, x_train_rev.shape

## Define Keras Model

In [ ]:
from keras.layers import Input, Dense, RepeatVector, LSTM, Conv1D
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences

In [ ]:
x_train_rev = pad_sequences(x_train_rev, maxlen=10, padding='post', truncating='post', value=0)

In [ ]:
inputs = Input(shape=x_train_att[0].shape)
outputs = Input(shape=x_train_rev[0].shape)

x = Dense(100, activation='relu')(inputs)
print(x)
x = Dense(1, activation='relu')(x)
print(x)
x = RepeatVector(10)(x)
print(x)
x = LSTM(1, return_sequences=True)(x)
print(x)
x = LSTM(1, return_sequences=True)(x)
print(x)
x = LSTM(1, return_sequences=True)(x)
print(x)

model = Model(inputs, x)
model.compile(optimizer='adam',
              loss='kullback_leibler_divergence',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train_att, x_train_rev, batch_size=32, epochs=50, verbose=1)

In [ ]:
x_train_att[10:20]

In [ ]:
model.predict(asarray([[1, 9,  4]]))